# DATA EXPLORATION

**Autor:** Carlos Fort Duart

**Fecha:** 21/11/2025

**Archivo de datos analizado:** retail_store_sales.csv de la plataforma Kaggle

**Objetivo:** Exploratory Data Analysis (EDA) detallado.

**Información del dataset:** Se trata de un dataset sobre ventas minoristas, extraído de la plataforma Kaggle. Se trata de una fuente fiable pero sintética y con errores intencionados. Se presentan datos transaccionales individuales. Este dataset tiene valores faltantes en varias columnas, necesita estandarización de fechas, posibles incoherencias matemáticas y requiere imputación y limpieza, lo que lo hace perfecto para este proyecto de EDA.

## 1. Preparación y carga del dataset

### 1.1. Importamos librerías y leemos el fichero

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats

# Ajustes de matplotlib para que los gráficos salgan de forma legible
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 10

### 1.2. Carga del dataset

In [ ]:
# Leemos el CSV
df = pd.read_csv("../data/retail_store_sales.csv")


1.3. Añadimos path para cargar funciones

In [ ]:
import sys
import os

# Añade la carpeta 'src' al path de Python
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Ahora podemos importar funciones.py
import funciones

## 2. Estructura del dataset

### 2.1. Dimensiones del dataset, variables y tipos

In [ ]:
from funciones import ver_estructura

In [ ]:
ver_estructura(df)

Shape (filas, columnas): (12575, 11)


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
5,TXN_7482416,CUST_09,Patisserie,NaN,NaN,10.0,200.0,Credit Card,Online,2023-11-30,NaN
6,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-store,2023-06-10,True
7,TXN_1372952,CUST_21,Furniture,NaN,33.5,NaN,NaN,Digital Wallet,In-store,2024-04-02,True
8,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1.0,27.5,Credit Card,In-store,2023-04-26,False
9,TXN_2722661,CUST_25,Butchers,Item_22_BUT,36.5,3.0,109.5,Cash,Online,2024-03-14,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


Notemos que el dataset tiene 12575 filas, donde cada fila es información de una transacción individual; y 11 columnas, cada una para una variable diferente.

A primera vista tenemos las siguientes variables:

**Transaction ID:** Identificador único de cada transacción. No es variable analítica, sino clave.

**Customer ID:** Identificador de cliente. Puede servir para análisis de recurrencia, CLV, etc.

**Category:** Categoría del producto (variable categórica). Importante para análisis de mix de ventas.

**Item:** Nombre del artículo. Puede tener alta cardinalidad.

**Price Per Unit:** Precio unitario. Variable numérica directa.

**Quantity:** Cantidad comprada. Numérica, puede relacionarse con volumen de compra.

**Total Spent:** Importe total de la compra. Debe relacionarse con Price Per Unit × Quantity.

**Payment Method:** Método de pago (categórica). Útil para análisis de cliente, tasas de adopción, etc.

**Location:** Ubicación de la tienda/venta. Variable categórica geográfica.

**Transaction Date:** Fecha de la transacción. Debería convertirse a datetime.

**Discount Applied:** Indica si se aplicó descuento. Puede ser categórica tipo “Yes/No”, numérica o incluso texto inconsistente.

Cada fila representa una transacción individual.

Las columnas incluyen:

- Identificadores (como **Transaction ID** y **Customer ID**)

- Información descriptiva del producto (como **Category** y **Item**)

- Variables económicas (como **Price Per Unit**, **Quantity** y **Total Spent**)

- Información contextual (como **Payment Method** y **Location**)

- Fecha (como **Transaction Date**)

- Campo opcional de descuento (como **Discount Applied**)

Vemos que los tipos detectados son:

- 8 columnas tipo object (texto/categóricas)
- 3 columnas numéricas (float64)

A primera vista podemos decir que la columna de fecha aún está como texto (object), por lo que será necesario convertirla a datetime.

## 3. Exploración

### 3.1. Resumen variables cuantitativas y cualitativas

In [ ]:
from funciones import resumen_num_cat


In [ ]:
resumen_num_cat(df)

=== RESUMEN VARIABLES NUMÉRICAS ===


,count,mean,std,min,25%,50%,75%,max,range
Price Per Unit,11966.0,23.365912,10.743519,5.0,14.0,23.0,33.5,41.0,36.0
Quantity,11971.0,5.536380,2.857883,1.0,3.0,6.0,8.0,10.0,9.0
Total Spent,11971.0,129.652577,94.750697,5.0,51.0,108.5,192.0,410.0,405.0



=== CARDINALIDAD (VALORES ÚNICOS) ===


,n_unique
Transaction ID,12575
Transaction Date,1114
Total Spent,228
Item,201
Price Per Unit,26
Customer ID,25
Quantity,11
Category,8
Payment Method,3
Discount Applied,3



=== TOP VALORES VARIABLES CATEGÓRICAS ===

Columna: Transaction ID — Unique: 12575 — Nulos: 0
Transaction ID
TXN_2407494    1
TXN_6867343    1
TXN_3731986    1
TXN_9303719    1
TXN_9458126    1
TXN_4575373    1
TXN_7482416    1
TXN_3652209    1
TXN_1372952    1
TXN_9728486    1
TXN_2722661    1
TXN_8776416    1
TXN_5422631    1
TXN_5874772    1
TXN_4413070    1
TXN_6295698    1
TXN_7239201    1
TXN_4397672    1
TXN_9743446    1
TXN_1434202    1

Columna: Customer ID — Unique: 25 — Nulos: 0
Customer ID
CUST_05    544
CUST_24    543
CUST_13    534
CUST_08    533
CUST_09    519
CUST_15    519
CUST_16    515
CUST_23    513
CUST_20    507
CUST_18    507
CUST_01    507
CUST_11    503
CUST_10    501
CUST_22    501
CUST_21    498
CUST_12    498
CUST_07    491
CUST_02    488
CUST_17    487
CUST_19    487

Columna: Category — Unique: 8 — Nulos: 0
Category
Electric household essentials         1591
Furniture                             1591
Food                                  1588
Milk Product

Además de los estadísticos media, desviación y cuartiles, encontramos el mínimo y el máximo.
- La variable **PricePerUnit** toma valores entre 5 y 41.
- La variable **Quantity** toma valores entre 1 y 10
- La variable **Total Spent** toma valores entre 5 y 410.

Comentamos brevemente las salidas.

- La variable **Transaction ID** tiene 12.575 valores únicos y 0 nulos, es decir, cada transacción tiene un ID único. Esto confirma que no existen duplicados y que la columna sirve como clave primaria perfecta.

- La variable **Customer ID** tiene 25 valores únicos y 0 nulos. Hay 12.575 transacciones realizadas por solo 25 clientes, lo que significa que en promedio cada cliente realiza 503 compras aproximadamente. Algunos clientes compran más que otros, ya que el cliente más activo (CUST_05) tiene 544 transacciones y el cliente menos activo (entre los mostrados) tiene entre 487–491.

- La variable **Category** tiene 8 valores únicos y 0 nulos. Las categorías son "Electric household essentials", "Furniture", "Food", "Milk Products",  "Butchers", "Beverages", "Computers and electric accessories" y "Patisserie". Observamos que las categorías están muy equilibradas ya que todas rondan entre 1528 y 1591 registros, lo cual indica que el dataset está bien balanceado entre tipos de productos y no hay una categoría claramente dominante. Las ventas parecen distribuidas de forma bastante homogénea.

- La variable **Item** tiene 201 valores únicos y 1213 nulos. Es la variable con mayor diversidad de productos, lo cual es normal en retail. Sin embargo, 1213 transacciones no registran item, solo categoría. Las posibles causas pueden ser: errores en el sistema de captura, artículos descatalogados, transacciones donde solo se registró categoría, ...

- La variable **Payment Method** tiene 3 valores únicos y 0 nulos. Se distribuyen en 4310 en la categoría "Cash", 4144 en la categoría "Digital Wallet" y 4121 en la categoría "Credit Card". Los tres métodos están muy equilibrados pero el efectivo sigue siendo ligeramente dominante.

- La variable **Location** tiene 2 valores únicos y 0 nulos. En concreto, la categoría "Online" con	6.354 y la "In-store" con	6.221. Ventas casi 50%–50% entre online y físico.

- La variable **Transaction Date** tiene 1114 valores únicos y 0 nulos. Notemos que todas las transacciones tienen fecha, lo que es muy útil para análisis temporal.Pero antes debe convertirse a tipo datetime.

- La variable **Discount Applied** tiene 3 valores únicos (True, False, NaN) y 4199 nulos: True	4219, NaN	4199 y False	415. El 33% de los datos no indican si hubo descuento. Lo más probable es que los nulos signifiquen que o bien no se aplicó descuento y no se registró, bien el campo se introdujo en una fecha posterior o bien hay una falta de consistencia en el sistema. Notemos que muchísimas más operaciones tienen True que False. Esto sugiere que cuando hay descuento, sí se registra y que cuando no lo hay, puede ser que muchas veces se dejó vacío.

### 3.2. Calidad de los datos (valores nulos, duplicados, incosistencias y outliers)


In [ ]:
from funciones import calidad_datos

In [ ]:
calidad_datos(df, numeric_cols=['Price Per Unit', 'Quantity', 'Total Spent'])

=== NULOS ===


,missing_count,missing_pct
Discount Applied,4199,33.391650
Item,1213,9.646123
Price Per Unit,609,4.842942
Total Spent,604,4.803181
Quantity,604,4.803181
Transaction ID,0,0.000000
Customer ID,0,0.000000
Category,0,0.000000
Payment Method,0,0.000000
Location,0,0.000000



=== DUPLICADOS ===
Filas duplicadas exactas: 0

=== INCONSISTENCIAS TOTAL SPENT (SI APLICA) ===
Inconsistencias detectadas: 0

=== VALORES NEGATIVOS (SI APLICA) ===
Filas con valores negativos: 0

=== OUTLIERS (IQR) ===
Price Per Unit: 0 outliers detectados
Quantity: 0 outliers detectados
Total Spent: 60 outliers detectados


Observamos que las variables **Transaction ID**, **Customer ID**, **Category**, **Payment Method**, **Location** y **Transaction Date** están completas y no tienen valores faltantes.

En cambio, las variables restantes sí que tienen valores nulos. Observamos que las variables **Total Spent** y **Quantity** tienen 604 valores faltantes, es decir, un 4,8% de las filas tienen estas columnas vacías. De forma similar, **Price Per Unit**, **Item** y **Discount Applied** tienen 609, 1213 y 4199 valores faltantes, respectivamente.


Observamos que no hay filas duplicadas.

Comprobamos que el total gastado es el precio por unidad por cantidad.

No hay inconsistencias matemáticas encontradas, es decir, el total está bien calculado como el producto del precio por unidad y la cantidad.

Comprobamos también que no haya ningún valor de Precio Por Unidad o Cantidad que sea negativo, ya que no tendrían sentido estos valores. Deben ser positivos.

No hay valores negativos.

Sobre la variable **Price Per Unit** no hay outliers. Esto indica que los precios por unidad están dentro de un rango razonable: no hay precios anormalmente altos, no hay precios sospechosamente bajos y la distribución de precios es homogénea y estable.

La variable **Quantity** tampoco presenta valores extremos. Esto significa que los valores de Quantity se concentran en un rango estrecho. El comportamiento de compra es uniforme entre productos y no hay compras extrañamente grandes.

**Total Spent** depende de dos variables. Aunque cada una individualmente no tenga outliers, la combinación de un precio alto y una cantidad alta puede producir valores extremos. Esto sí que tiene sentido lógico.